# Cleaning data

**Author:** 'Felipe Millacura'
    
**Date:** '14th February 2021'

### Learning objectives

* Understand the importance of cleaning data before any analysis
* Know the common errors that crop up
* Be able to deal with common errors (apart from outliers and missing values - covered next)



# Data Cleaning 

It is said that the average data analyst spends about 80% of their time cleaning their data. That's a long time! And it highlights how important it is to understand data cleaning. Today's lessons will teach you how to do some important data cleaning in Python. 



![](http://blog.appliedinformaticsinc.com/wp-content/uploads/2015/08/Screen-Shot-2015-08-13-at-4.43.50-PM-225x300.png)


Exactly what data cleaning entails depends a lot on what the data you’ve got looks like, and what you’re trying to do with it. But often, you’ll need to look through it to remove irrelevant or erroneous data points. You may need to create new variables, or join multiple data sets together to get what you need for your analysis. And you’ll also have to determine how to handle missing values, as very few real-world datasets are going to be 100% complete. You'll then have to get it into the right format, sort it, and summarise it. Lots of steps along the way!


In this first section, we will begin with the data cleaning you do as you load your data in, BEFORE you start looking at any odd values or analysing anything. The next sections will cover the more complex data cleaning issues of outliers and missing values that are more complex to deal with. 


Today we will be working with a sample dataset from the gaming company Steam. The `video_game` dataset contains information on what games some Steam users bought and how many hours they spent playing them. 

This dataset is a list of user behaviors, with columns:   

*  `customer id`: player id    
*  `game name`: game name    
*  `what`: what activity were they playing   
*  `value`: hours of game play   
*  `bought online`: 1 if the user bought the game online, and a 0 if they didn't.  

[You can find the full dataset here](https://www.kaggle.com/tamber/steam-video-games)


### Removing Metadata

Metadata is data about data. Often this will be listed in an accompanying file. However, sometimes you'll find that there is metadata within the data file as well, and you don't want it to be there. In our example dataset, there are 4 rows at the top, which just describe what the data is, and when it was last modified. What happens when you try and load this? 


### Gather

In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv("data/steam-200k-sample.csv")

You get an error...   
 
```python
UnicodeDecodeError: 'utf-8' codec can't decode byte 0xb4 in position 1: invalid start byte
```

This is because it can't read the metadata at the top, as there are some non `utf-8` characters:

![](images/nonutf8.png)


We can solve this by adding the `encoding = 'unicode_escape'` argument while importing our file

In [ ]:
df = pd.read_csv("data/steam-200k-sample.csv", encoding = 'unicode_escape')

Now it worked! Let's see what we got

In [ ]:
df.head()

Even though the data was imported this time, there seem to be some issues with the first four rows. 

The solution here is to add the `skiprows` argument into your `pd.read_csv` call, allowing you to remove the metadata at the top.

In [ ]:
df = pd.read_csv("data/steam-200k-sample.csv", skiprows = 4, encoding = 'unicode_escape')

Now we can check again

In [ ]:
df.head()

## Assess

We can retrieve the basic info of our DataFrame using `.info()`

In [ ]:
df.info()

#### Checking variable types

We touched briefly on ensuring that Python reads in our variables correctly. Look at your column specification above. Has Python read in all your data correctly?

We can see that the `bought online` column has been read in as a `int64`, but from our data description we know this should be a `bool`. Let's fix that just now. 


In [ ]:
df['bought online']

Ups! Wait a minute! Why can't we select the column? If we check our CSV we would see that there is an additional space at the end of the variable name! 

In [ ]:
df['bought online ']

#### Adding and fixing variable names

In an ideal world, you'll recieve data that is properly labelled. However in the real world, this isn't always the case. Let's look at what variable names we have in our dataset.   


In [ ]:
df.columns

Now we should fix any variable name presenting issues. For that we need to use `rename()`. Find further information [here](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.rename.html)

In [ ]:
df_clean = df.rename(columns={'bought online ': 'bought_online',
                   'customer id': 'customer_id',
                   'game name': 'game_name',
                   'what': 'activity'})

df_clean

Now let's comeback to changing the `bought_online` data type by using `astype()`. Read more about it [here](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.astype.html)

In [ ]:
df_clean['bought_online'] = df_clean['bought_online'].astype('bool')

df_clean

Ok great, we now know we've got the right data coming in! But is there a way to automate this process?

In [ ]:
!pip install pyjanitor

In [ ]:
import janitor

In [ ]:
df_clean_jan = df.clean_names()
df_clean_jan.head()

The benefit here is that it is quick and harder to make errors. The downside is that you don't have control over the naming like you do if you rename everything manually. You can read more interesting features from `pyjanitor` [here](https://pyjanitor.readthedocs.io/notebooks/pyjanitor_intro.html#Example-analysis-of-the-data)

Now we can use our dataframe as usual

In [ ]:
df_clean.describe().transpose()

and work directly by using column names

In [ ]:
df_clean.bought_online.value_counts()

### Checking string variables 

Another common problem you will come across in data is `string` consistency. That is, when you have character variables, which represent categories, it is important that they are all coded the same. For example, let's look at the games **Assassin's Creed II** and **Left 4 Dead 2**.   


In [ ]:
games = ["Assassin's Creed II", "Left 4 Dead 2", "assassin's creed II", "left 4 dead 2"]

df_clean[df_clean.game_name.isin(games)]


Here you can see that we have the same game, listed with different case letters. One way of dealing with this is to make your string data all lowercase or uppercase. This is known as `defensive programming` - if you code in to always make string variables one case, then your code will usually run with conditional statements set. 


In [ ]:
# set strings to lowercase 

df_clean.game_name.str.lower()

In [ ]:
df_clean.game_name.str.upper()

In [ ]:
df_clean.game_name = df_clean.game_name.str.upper()

Great. Now we know that if we end up summarising the data by categories, there won't be any case issues. 



### Removing useless columns 

The final common issue we will cover is redundant columns. Sometimes you receive variables which don't contain any useful data. For example, in our dataset we have the `activity` variable, which contains the entry `play`. 

We can check whether the column contains only one value by using the `unique` function. More information [here](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.unique.html)


In [ ]:
df_clean.activity.unique()

In this case, we have only one value in the `activity` column, and it isn't really adding anything to the data. So let's `drop` it. Read more [here](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.drop.html)


In [ ]:
df_clean.drop(columns=['activity'])


And there we go. We've covered the inital data cleaning process! Next, we will learn more about outliers and missing data points. 



### Recap 


**What are the main data cleaning tasks you might perform before analysis?**

Checking the data is loaded in correctly, checking variable names, checking variable types, checking strings are consistent, and removing redundant columns.   


**Which of the following is an example of a preferred variable name?**

     user id   
     user_id  
     User_Id  


The second one, user_id would be preferred. It is all one case, and there are no spaces. This means it is easier to read, and less prone to errors in capitalisation (remember Python is case sensitive language) 


# Dealing with outliers

### Learning objectives

* Understand the implications of outliers in a dataset
* Be able to identify outliers in a dataset and decide relevant way forward
* Understand the options available deal with outliers



## Outliers


#### What is an outlier? 

An outlier is an observation (or set of observations) which appear to be inconsistent with the dataset. For example, any datapoint that is either very high, very low, or just unusual (within the context of your project), is an outlier. They may be indications of data quality issues. As part of data cleansing, a data scientist/analyst would typically identify and then address the outliers. 

What do outliers mean? It's hard to say. Some culprits:

1. An outlier may be due to variability in the measurement 
2. Maybe indicate experimental error - the data might have been collected wrongly or there is equipment error
3. Maybe the data itself is wrong: this happens a lot when you collect data from people. They miss out responses, or give wrong responses and it can affect the data. 
4. Might just be a genuine extreme data point - it happens!


But it's important to identify them and deal with them, as an outlier can cause serious problems in statistical analyses.


#### How do we identify an outlier?

Often the easiest and quickest way to identify an outlier is to plot your data. Usually outliers are shown on boxplots (or sometimes histograms and scatterplots depending on what data you have). 

A boxplot is a standardized way of displaying the distribution of data based on a five number summary (“minimum”, first quartile (Q1), median, third quartile (Q3), and “maximum”). It can tell you about your outliers and what their values are. It can also tell you if your data is symmetrical, how tightly your data is grouped, and if and how your data is skewed.


![](https://miro.medium.com/max/18000/1*2c21SkzJMf3frPXPAR_gZA.png)


Let's have a look now. We can use the built in dataset `iris` for this. Let's do a boxplot first:


In [ ]:
from sklearn import datasets

In [ ]:
iris = datasets.load_iris()
print(iris)

In [ ]:
df_iris=pd.DataFrame(iris.data,columns=iris.feature_names)
df_iris= df_iris.clean_names()
df_iris.head()

In [ ]:
import plotly.express as px

In [ ]:
px.box(df_iris)

Looks like we have outliers in the `sepal_width_cm_` variable. Let's plot this more


So, what does this tell us? In our boxplot we can see we have some outlier values: some flowers have really low and high sepal widths, compared to the others. 

Aside from just having a look visually, outliers can be defined mathematically. In mathematical sense, an outlier is usually defined as an observation more than three standard deviations from the mean (although sometimes you'll see 2.5 or 2 as well). It's not the only mathematical definition, but since it's the one that's used the most often that's what we'll use here as well.

Z-score: The number of standard deviations away from the mean that a particular observation is. (You can see the mathematical definition [here](https://en.wikipedia.org/wiki/Standard_score), if you're curious.) A negative Z-score means an observation is below the mean, while a positive one means it above it. The further away from 0 the Z-Score is, the further away from the mean your observation is.

<img src="https://upload.wikimedia.org/wikipedia/commons/thumb/2/25/The_Normal_Distribution.svg/1280px-The_Normal_Distribution.svg.png"/>


## What do we do with them once we've identified them? 

There are some generally accepted methods for dealing with outliers: 

**Delete the outlier values or even the actual variable where the outliers exist** : This is probably the easiest option for dealing with outliers, but is also quite dangerous as it is only a valid options if the outliers aren't **real** data values.

**Transform the values or the variable itself** : in some cases, you may be justified in imputing the outlier values with some other value. For example, maybe someone accidently shifted a decimal place when entering the data. If you know this is the case, then you could reasonably replace this value with say - the median value of the data. The end result is the same: the outliers are artificial, as is the value you've replaced it with. 

**Split the outliers into their own dataset** : this can be a good approach when you're interested in WHY there are outliers. Are there patterns in the data? Is there something in it that can explain the presence of the values? For example, perhaps you have length data recorded for a variable. Maybe the outlier values are all those measured in cm, and the rest have been put in in inches by acccident? Splitting the data could help uncover this. 

Overall, there are many different ways of dealing with outliers, and it's up to you to make the right call on which to use (or which combination of methods to use). The important point is to record and justify your decision around which method you have chosen to use.



<blockquote class='task'>
<b>Task - 3 mins </b>

What would the pitfalls of each approach mentioned above be?
<br><br>
<details>
<summary><b>Answer</b></summary>

<b>* Deleting outliers:</b> you could be deleting real data that might be there for a valid reason!

<b>* Transforming data:</b> It can be difficult to find out whether an outlier is there by genuine mistake, and again, you need to be careful that you're not replacing real data.  
<b>* Splitting and investigating:</b> this can be time consuming, and often you're not overly interested in what causes outliers so it's not worth the time spent investigating. Of course, there might only be a couple of outliers to deal with and so making a whole new dataset is a bit redundant. 

</details>
</blockquote>

## Dealing with outliers in a dataset: Brazil's House of Deputies Expenditures

To pratice identifying outliers we will use a dataset showing the expense accounts from members of Brazil's House of Deputies. The data is open source, and [free to download from the official government site](http://www2.camara.leg.br/transparencia/cota-para-exercicio-da-atividade-parlamentar/dados-abertos-cota-parlamentar). However, we will use [the version from kaggle](https://www.kaggle.com/epattaro/brazils-house-of-deputies-reimbursements/) which has been converted to csv files and the titles converted to English for us. 

This dataset is a good one because (a) it contains a lot of outliers, and (b) the outliers are actually data points you should be interested in. If a government official is getting a lot of reimbursements, it means they are spending a huge amount of public funds, and that's probably something we want to pay attention to.

The `refunds_info` dataset contains expenses data and amounts for deputies from 2009 to 2017, and info about who is claiming the expenses.



In [ ]:
df_refunds  = pd.read_csv("data/refunds_info_sample.csv", encoding = 'unicode_escape').clean_names()
df_refunds.info()

## Identifying outliers

One of the easiest ways to see if your dataset has outliers is to plot your data. The type of plot you pick will depend on how many and what kind of variable you're interesting in looking at. Today, we're going to look at three different plots.

### Using a boxplot 
In statistics, a boxplot is a simple way to gain information regarding the shape, variability, and centre (or median) of a statistical dataset, so we'll use the boxplot with our data to see if we can identify both the median and if there are any outliers. 

Let's first use a boxplot to see if we have any outliers in the refunds that politicians are getting. 


In [ ]:
px.box(df_refunds.refund_value)

WOW! We have a lot of high values. That line is actually the boxplot: everything above that line are a bunch of points showing outliers that are overlapping with each other. What might be better to help us understand where these outliers are in the data is for us to break this up by a category. 


In [ ]:
px.box(df_refunds,
       x='refund_value',
       y='refund_description',
       color = 'refund_description',
       orientation = 'h').update_layout(showlegend=False) 

This tells us more! Mainly, that there appears to be a lot of outliers in the dissemination of parlimentary activity and mail categories, but not so many in fuel and phone costs.   


<blockquote class='task'>
<b>Task - 5 mins</b>

Make a boxplot which shows whether certain political parties have more refund outliers. 

<details>
<summary><b>Answer</b></summary>

```python
px.box(df_refunds,
       x='refund_value',
       y='political_party',
       color = 'political_party',
       orientation = 'h').update_layout(showlegend=False) 
```

They all look pretty bad... more investigation needed!

</details>
</blockquote>



So far we've covered looking at outliers in a one variable (univariate outliers), and when we consider a categorical and numerical variable together (multivariate).   
Another way to look at outliers is to consider two numerical variables together (again, multivariate outliers). Sometimes this can help us understand outliers a bit better. In this case, it's better to use a scatterplot to check for outliers.   

For example, perhaps refunds are just higher for parties that have more members? Let's check this out. 


In [ ]:
px.scatter(df_refunds,
       x='party_nmembers',
       y='refund_value',
       color = 'political_party',
       orientation = 'v').update_layout(showlegend=False) 


Here we can see there is a really strong relationship between number of party members and `refund_value`: in this case, all the politicians expenses are associated with a specific number of party members (hence the straight line). You can also see that there are outliers across parties of all size.



## Identifying outliers 

Alright, now we know some techniques for visualising our outliers, let's learn how to identify which rows in our dataframe contain outliers. As you may remember from above, one way to identify outliers is to determine which points have a z-score that's far from 0. 

We can use the `zscore` function from the [Scipy package](https://docs.scipy.org/doc/scipy-0.14.0/reference/generated/scipy.stats.mstats.zscore.html) to quickly calculate the z-score for every value in a specific column in our table. 


In [ ]:
from scipy.stats import zscore

In [ ]:
refund_zscores = df_refunds[['party_nmembers', 'refund_value','tax_rebate']].apply(zscore)
refund_zscores


Now we just need to look and see which of those values are outside of our threshold. As we said before, mathematically an outlier is usually defined as an observation three standard deviations above or below the mean. So, let's try this. 



In [ ]:
is_outlier = (refund_zscores['refund_value'] > 3) | (refund_zscores['refund_value'] < -3)

In [ ]:
is_outlier.head()

In [ ]:
df_refunds['is_outlier'] = is_outlier

df_refunds

Here, what we have done is create a new column that's `True` if a value is an outlier (i.e. if it has a zscore of 3 or -3 in our outliers vector) and "FALSE" if it's not. Then we add it into the `refunds` data as a new `bool` column called `is_outlier`. Now we can use that new `is_outlier` column to plot the outlier and non-outlier values separately, and to separate and clean up the data. 


In [ ]:
px.box(df_refunds,
       x='refund_value',
       y='refund_description',
       facet_col = 'is_outlier',
       color = 'refund_description',
       orientation = 'h').update_layout(showlegend=False) 

You can see the outliers are so numerous, that they actually almost form their own datasets. This is a sign that these outliers aren't random - there is a pattern and there is potentially a reason for it. In this case, it's probably dirty politicians, so it's interesting to separate them out.   


### What to do once you've found them 


Ok, now that we know which specific values are outliers... what do we do with them? There are a lot of different strategies for what to do with outliers, and which one you should use for your specific project depends on what your goals are, and why you think you have outliers. Let's look at them all. 


**Option 1 : Removing them**
The easiest option for handling outliers is just to drop the rows that have outliers in them.



In [ ]:
refunds_no_outliers = df_refunds[df_refunds['is_outlier'] != True]

In [ ]:
refunds_no_outliers

In [ ]:
px.box(refunds_no_outliers,
       x='refund_value',
       y='refund_description',
       color = 'refund_description',
       orientation = 'h').update_layout(showlegend=False) 

Looking better! Although now we've got rid of the main outliers in one, you can see that the other columns before, which looked alright, now seem to have a lot of outliers as well. Sigh... This is usually a long process. You do one category, then look at another. And do one variable, and then look at another. 

**Option 2 : Consider outliers separately** 

This option is the most lengthy, and is the one we've already covered - setting the outliers into a separate table and plotting them separately. As we've already discussed, in this particular case, this is probably the best option, given that it is interesting and there is a clear non-random reason for the data.   



**Option 3: Replace the outliers with imputations**   

As discussed in the intro, this option is good when you know for sure you have data entry errors. It can also be good if there is a clear pattern of outliers - e.g. all `NaN`'s are -9999. So, what is imputation?

<br>
<center><b>Imputation:</b> The fancy math term for guessing what the value of a missing cell in your dataset should actually be. </center>

So, for example, we can replace all the outliers with `NaN`'s using `where` (more info [here](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.where.html))


In [ ]:
import numpy as np

In [ ]:
df_refunds['refund_value'].where(~df_refunds['is_outlier'])

Yep, lots of `NaN`'s in there, so we can see it worked. In some cases, it might be a good idea to replace the outlier values with a value such as the median. However, in this case, it's probably not a good idea to replace the outlier values with another value, as the outlier values themselves are of interest to us. 


<blockquote class='task'>
<b>Task - 5 mins</b>

Instead of replacing all the outlier values with NaN's, replace them with the median. Check if you did it by calculating a summary stat. 

<br>
<details>
<summary><b>Answer</b></summary>
    
```python    
    
df_refunds['refund_value'].where(~df_refunds['is_outlier'], other= df_refunds['refund_value'].median()
    
``` 
</details>
</blockquote>


Great. Now you've managed to work with some outliers from one column of the dataset. As you can see, outlier identification and removal is a long and laborious process, that - depending on the dataset you are working with - consumes a lot of your time. 


## Recap

<br>

What is the mathematical definition of an outlier?

<br>

<details>
<summary><b>Answer</b></summary>
A value that is more than 3 standard deviations outside the mean.
</details>
<br>

What are two ways of identifying outliers?

<br>
<details>
<summary><b>Answer</b></summary>
Visualise them or calculate a zscore. 
</details>
<br>

What are three methods of dealing with outliers once you find them?

<br>
<details>
<summary><b>Answer</b></summary>
Delete them, impute them, separate them out into a new dataset.   
</details>
<hr>